""""

1. Tried pulling directly from the website source api, got my ass kicked
2. Skywalk API is working and they fixed some requests
3. Testing dashboard combined with Apartments.com data-puller

""""

In [1]:
import requests
from bs4 import BeautifulSoup
import json

appfolio_api_key = "wc9DeJ9UHoJKkEsgr9CN4W.8AnBp5DgrPE2HNdNMQ1HxwDlBkJnYRZDyGvletEznkE"
payload = {"X-API-KEY" : appfolio_api_key}
params = { "search": "?propertyId=51"}
r = requests.get('https://api.skywalkapi.com/v1/balance-sheet', headers=payload, params = params)
r.raise_for_status()

In [2]:
json.loads(r.content)

{'meta': {'status': 'updating',
  'path': '/v1/balance-sheet',
  'lastUpdated': None,
  'jobId': 'a1eb4a11-88a2-495f-947d-1dbef418eaf3',
  'query': {'accounting': 'cash', 'date': '02/22/2022'}}}

# Dashboard

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

import ApartmentsScraperHelper

bb = "xtqrw9_u9Isz8svW"
# pull the same code everytime but for San Marco
ApartmentsScraperHelper.main(bb = bb)

pd.read_csv(bb+".csv")

NameError: name 'csv' is not defined

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html
import pandas as pd

#zillow rent data
rent_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv?t=1644973146")
rent_df.drop(columns= ["RegionID", "SizeRank"], inplace=True)
rent_df.head()

,RegionName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,United States,1294.0,1299,1305.0,1310,1315,1321,1326,1331,1337,...,1708,1729,1750,1771,1793,1815.0,1837.0,1859,1881,1904
1,"New York, NY",2256.0,2266,2276.0,2286,2296,2306,2316,2326,2336,...,2526,2556,2586,2617,2648,2680.0,2712.0,2746,2780,2814
2,"Los Angeles-Long Beach-Anaheim, CA",1792.0,1803,1814.0,1825,1835,1846,1857,1868,1878,...,2497,2524,2550,2577,2604,2631.0,2659.0,2687,2716,2744
3,"Chicago, IL",1411.0,1415,1419.0,1423,1426,1430,1433,1437,1440,...,1649,1662,1675,1688,1702,1716.0,1729.0,1744,1758,1772
4,"Dallas-Fort Worth, TX",1128.0,1132,1135.0,1139,1143,1146,1150,1154,1157,...,1534,1556,1578,1600,1622,1645.0,1667.0,1690,1714,1737


In [61]:
reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")
reframed_rent_df

,RegionName,Date,ZRent
0,United States,2014-01,1294.0
1,United States,2014-02,1299.0
2,United States,2014-03,1305.0
3,United States,2014-04,1310.0
4,United States,2014-05,1315.0
...,...,...,...
9880,College Station TX,2021-09,1524.0
9881,College Station TX,2021-10,1532.0
9882,College Station TX,2021-11,1541.0
9883,College Station TX,2021-12,1550.0


In [76]:
# housing from ZRent
housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1645574925")
housing_df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)

reframed_housing_df = pd.DataFrame(housing_df.set_index("RegionName").stack()).reset_index()
reframed_housing_df.columns = ["RegionName", "Date", "ZEstimate"]
reframed_housing_df.RegionName = reframed_housing_df.RegionName.str.replace(",", "")
reframed_housing_df.Date = reframed_housing_df.Date.str[:-3]

reframe_df = pd.merge(reframed_rent_df, reframed_housing_df, how = "left", left_on = ["RegionName", "Date"], right_on = ["RegionName", "Date"])

reindexed_reframed_df = reframe_df.set_index(["RegionName", "Date"])

# adding percentage change
reindexed_reframed_df["rent_pct_change"] = reindexed_reframed_df["ZRent"].pct_change()
reindexed_reframed_df["home_value_pct_change"] = reindexed_reframed_df["ZEstimate"].pct_change()


reindexed_reframed_df.reset_index(inplace = True)
reindexed_reframed_df = reindexed_reframed_df[reindexed_reframed_df.Date != "2014-01"]

reindexed_reframed_df

In [97]:
app = JupyterDash(__name__)
del app.config._read_only["requests_pathname_prefix"]
server = app.server


template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reindexed_reframed_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZRents'),

        dcc.Graph(
            id='Zrent',
        )]
    
     ,style=style),
    
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZRents Change'),
        dcc.Graph(
            id='Zrent_pct_change',
        )]
    
     ,style=style  ),
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZEstimates'),
        dcc.Graph(
            id='ZEst',
        )]
    
       ,style=style             ),
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZEstimates Change'),
        dcc.Graph(
            id='ZEst_pct_change',
        )]
        ,style=style  
                  ),

    ])

@app.callback(
    Output('Zrent', 'figure'),
    Output('Zrent_pct_change', 'figure'),
    Output('ZEst', 'figure'),
    Output('ZEst_pct_change', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reindexed_reframed_df[reindexed_reframed_df.RegionName.isin(selected_cities)]   
    fig1 = px.line(filtered_df, x="Date", y="ZRent", color="RegionName", hover_name="RegionName")
    fig1.update_layout(transition_duration=500, template = template)

    
    fig2 = px.line(filtered_df, x="Date", y="rent_pct_change", color="RegionName", hover_name="RegionName")
    fig2.update_layout(transition_duration=500, template = template,)
    
    fig3 = px.line(filtered_df, x="Date", y="ZEstimate", color="RegionName", hover_name="RegionName")
    fig3.update_layout(transition_duration=500, template = template)
    
    fig4 = px.line(filtered_df, x="Date", y="home_value_pct_change", color="RegionName", hover_name="RegionName")
    fig4.update_layout(transition_duration=500, template = template)
    
    return fig1, fig2, fig3, fig4
    
if __name__ == '__main__':
    app.run_server(debug=False)

127.0.0.1 - - [22/Feb/2022 19:06:33] "GET /_shutdown_7991f704-c2aa-4bed-8792-9a5cae4ba8c0 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Feb/2022 19:06:34] "GET /_alive_7991f704-c2aa-4bed-8792-9a5cae4ba8c0 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [22/Feb/2022 19:06:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:06:35] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL']


### Add apartment list too

In [3]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html
import pandas as pd


### zillow rent data
rent_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv?t=1644973146")
rent_df.drop(columns= ["RegionID", "SizeRank"], inplace=True)

reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")


### Zilllow housing
housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1645574925")
housing_df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)

reframed_housing_df = pd.DataFrame(housing_df.set_index("RegionName").stack()).reset_index()
reframed_housing_df.columns = ["RegionName", "Date", "ZEstimate"]
reframed_housing_df.RegionName = reframed_housing_df.RegionName.str.replace(",", "")
reframed_housing_df.Date = reframed_housing_df.Date.str[:-3]

### Combining data frames
reframe_df = pd.merge(reframed_rent_df, reframed_housing_df, how = "left", left_on = ["RegionName", "Date"], right_on = ["RegionName", "Date"])
reindexed_reframed_df = reframe_df.set_index(["RegionName", "Date"])

### adding percentage change
reindexed_reframed_df["rent_pct_change"] = reindexed_reframed_df["ZRent"].pct_change()
reindexed_reframed_df["home_value_pct_change"] = reindexed_reframed_df["ZEstimate"].pct_change()

# removing first month because of overlap issue
reindexed_reframed_df.reset_index(inplace = True)
reindexed_reframed_df = reindexed_reframed_df[reindexed_reframed_df.Date != "2014-01"]

### PLOTTING ###
app = JupyterDash(__name__)
del app.config._read_only["requests_pathname_prefix"]
server = app.server

template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reindexed_reframed_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZRents'),

        dcc.Graph(
            id='Zrent',
        )]
    
     ,style=style),
    
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZRents Change'),
        dcc.Graph(
            id='Zrent_pct_change',
        )]
    
     ,style=style  ),
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZEstimates'),
        dcc.Graph(
            id='ZEst',
        )]
    
       ,style=style             ),
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZEstimates Change'),
        dcc.Graph(
            id='ZEst_pct_change',
        )]
        ,style=style  
                  ),

    ])

@app.callback(
    Output('Zrent', 'figure'),
    Output('Zrent_pct_change', 'figure'),
    Output('ZEst', 'figure'),
    Output('ZEst_pct_change', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reindexed_reframed_df[reindexed_reframed_df.RegionName.isin(selected_cities)]   
    fig1 = px.line(filtered_df, x="Date", y="ZRent", color="RegionName", hover_name="RegionName")
    fig1.update_layout(transition_duration=500, template = template)

    
    fig2 = px.line(filtered_df, x="Date", y="rent_pct_change", color="RegionName", hover_name="RegionName")
    fig2.update_layout(transition_duration=500, template = template,)
    
    fig3 = px.line(filtered_df, x="Date", y="ZEstimate", color="RegionName", hover_name="RegionName")
    fig3.update_layout(transition_duration=500, template = template)
    
    fig4 = px.line(filtered_df, x="Date", y="home_value_pct_change", color="RegionName", hover_name="RegionName")
    fig4.update_layout(transition_duration=500, template = template)
    
    return fig1, fig2, fig3, fig4
    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Feb/2022 19:10:29] "GET /_alive_a9dfec7a-0208-445d-a55e-e0d520905064 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [22/Feb/2022 19:10:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2022 19:10:37] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


selected_cities ['United States', 'Jacksonville FL']


127.0.0.1 - - [22/Feb/2022 19:10:37] "POST /_dash-update-component HTTP/1.1" 200 -
